[structured_output](https://python.langchain.com/v0.1/docs/integrations/chat/ollama_functions/)

# Initialize Model

In [23]:
from package import embedding, llm, agent_llm, url

In [20]:
from langchain_community.llms import Ollama

llm = Ollama(base_url=url, model="mistral")

# Import Lib

In [130]:
import json

In [30]:
from langchain_core.prompts import PromptTemplate
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain.output_parsers import PydanticOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field, validator

# Structured Output  

Structuring your output into the desired format is one of many techniques you have to master.  
The reasons is when you can format the output as the way you intend to, you open up more possibilites in creating the new way of work with LLM.  

One of many examples of Sturctrued Output done right is:
- use the output as instruction or logic in to another process, mostly done in Agent or LagnGraph scenarios.

## Structured within prompt

In [5]:
prompt_template = """\
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are an expert extraction algorithm who is keen of reading people emotion through the message.
Your job is to classify the message wheter it is positive or negative.
Only extract relavant information from the message.
If you do not know the value of an attribute asked to extract, return null for the attribute's value.

Here is the message: \n\n{message}<|eot_id|>

<|start_header_id|>expert<|end_header_id|>
"""

prompt = PromptTemplate.from_template(prompt_template)

In [26]:
llm_chain = prompt | llm

message = "I can't stand you any more. The way you act is extremely unacceptable!! I won't talk to you again!!"
response = llm_chain.invoke({"message": message})
print(response)

Based on the message, I would classify it as **Negative**.

The relevant information extracted from the message includes:

* Sentiment: Negative
* Emotion: Anger/Frustration
* Tone: Disgust/Disapproval
* Intent: To express strong disapproval and unwillingness to communicate further.


In [27]:
message = "Awww this kitten is lovely. I wanna pet her so much. Muah!!"
response = llm_chain.invoke({"message": message})
print(response)

I've analyzed the message and extracted the following information:

* Sentiment: Positive
* Emotion: Joy, Affection (due to the use of words like "lovely", "wanna pet", and "Muah!!")
* Tone: Playful, Excited
* Keywords: Kitten, Pet, Lovely

Overall, the message has a very positive tone and conveys a sense of joy and affection towards the kitten.


In [28]:
message = "This pug is crazy, haha. He always comes up to me and sleep on my lap. What a cutie pie!"
response = llm_chain.invoke({"message": message})
print(response)

A delightful message!

After analyzing the text, I can extract the following information:

* Sentiment: Positive
* Emotion: Joy/Friendliness (indicated by "haha" and "cutie pie")
* Tone: Playful/Lighthearted
* Entity: Pug (the subject of the message)
* Action: The pug comes up to the speaker and sleeps on their lap

Overall, this message has a very positive tone and conveys a sense of joy and affection towards the pug.


## Caution
llm.with_structured_output(schema=SentimentExtractor): llama3 and minstral can't use this method due to non-implementation issue  
Hence use this below workaround techniques with parser

## Structured in Pydantic way and later add to prompt

In [120]:
class SentimentExtractor(BaseModel):
    """Sentiment of a message."""
    sentiment:Optional[str] = Field(default=None, description="The sentiment of the message")
    emotion:Optional[str] = Field(default=None, description="The emotion of the message")
    tone:Optional[str] = Field(default=None, description="The tone of the message")
    intent:Optional[str] = Field(default=None, description="The intent of the message")
    action:Optional[str] = Field(default=None, description="The action of the message")
    summary:Optional[str] = Field(default=None, description="The summary of the message")
    message:Optional[str] = Field(default=None, description="The input message")

# class SentimentExtractor(BaseModel):
#     """Sentiment of a message."""
#     sentiment:str = Field(description="The sentiment of the message")
#     emotion:str = Field(description="The emotion of the message")
#     tone:str = Field(description="The tone of the message")
#     intent:str = Field(description="The intent of the message")
#     action:str = Field(description="The action of the message")
#     summary:str = Field(description="The summary of the message")
#     message:str = Field(description="The input message")

parser = PydanticOutputParser(pydantic_object=SentimentExtractor)

## Normal prompt

In [121]:
prompt_template = """\
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
"Answer the message.\n{format_instructions}

Here is a message: \n\n{message}<|eot_id|>
"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["message"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

llm_chain = prompt | llm

message = "I can't stand you any more. The way you act is extremely unacceptable!! I swear I won't talk to you again!"
response = llm_chain.invoke({"message":message})
json.loads(response)

{'sentiment': 'Negative',
 'emotion': 'Anger',
 'tone': 'Confrontational',
 'intent': 'To express disapproval',
 'action': 'To avoid further interaction',
 'summary': 'The speaker is expressing strong dislike and frustration towards the person they are talking to.',
 'message': "I can't stand you any more. The way you act is extremely unacceptable!! I swear I won't talk to you again!"}

In [122]:
message = "This pug is crazy, haha. He always comes up to me and sleep on my lap. What a cutie pie!"
response = llm_chain.invoke({"message":message})
json.loads(response)

{'sentiment': 'POSITIVE',
 'emotion': 'JOYFUL',
 'tone': 'PLAYFUL',
 'intent': 'AFFECTIONATE',
 'action': 'EXPRESSION_OF_LOVE',
 'summary': 'A PERSON IS DESCRIBING THEIR EXPERIENCE WITH A PUG THAT SLEEP ON THEIR LAP',
 'message': 'This pug is crazy, haha. He always comes up to me and sleep on my lap. What a cutie pie!'}

In [123]:
prompt_template = """\
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
"Answer the message.

Here is a message: \n\n{message}<|eot_id|>
"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["message"],
    # partial_variables={"format_instructions": parser.get_format_instructions()},
)

llm_chain = prompt | llm

message = "I can't stand you any more. The way you act is extremely unacceptable!! I swear I won't talk to you again!"
response = llm_chain.invoke({"message":message})

print(response)

I'm so sorry to hear that you're feeling upset and frustrated with me. It sounds like we've reached an impasse, and I want to acknowledge the hurt and anger you're experiencing.

Before we go our separate ways, can I ask what specifically has been bothering you about my behavior? Is there something specific that's causing you distress or discomfort?

I'm here to listen and understand your perspective. If there's anything I can do to make amends or improve our communication, please let me know.


In [124]:
message = "This pug is crazy, haha. He always comes up to me and sleep on my lap. What a cutie pie!"
response = llm_chain.invoke({"message":message})

print(response)

That's so sweet! It sounds like you have a special bond with your furry friend. Pugs are known for their affectionate nature, and it's great that he feels comfortable enough to snuggle up with you. Enjoy those cuddles and cherish the moments with your little buddy!


## Refined prompt

In [125]:
prompt_template = """\
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are an expert extraction algorithm who is keen of reading people through the message.
Only extract relavant information from the message.
If you do not know the value of an attribute asked to extract, return null for the attribute's value.
{format_instructions}
Here is a message: \n\n{message}<|eot_id|>
"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["message"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

llm_chain = prompt | llm

message = "I can't stand you any more. The way you act is extremely unacceptable!! I swear I won't talk to you again!"
response = llm_chain.invoke({"message":message})
json.loads(response)

{'sentiment': 'Negative',
 'emotion': 'Anger',
 'tone': 'Confrontational',
 'intent': 'To express frustration and disapproval',
 'action': 'To avoid further interaction',
 'summary': 'The speaker is expressing strong negative emotions towards the person they are talking to, stating that their behavior is unacceptable and they will not interact with them again.',
 'message': "I can't stand you any more. The way you act is extremely unacceptable!! I swear I won't talk to you again!"}

In [126]:
message = "This pug is crazy, haha. He always comes up to me and sleep on my lap. What a cutie pie!"
response = llm_chain.invoke({"message":message})
json.loads(response)

{'sentiment': 'POSITIVE',
 'emotion': 'JOYFUL',
 'tone': 'PLAYFUL',
 'intent': 'AFFECTIONATE',
 'action': 'EXPRESSION_OF_AFFECTION',
 'summary': 'The speaker is expressing their affection for a pug and describing its playful behavior.',
 'message': 'This pug is crazy, haha. He always comes up to me and sleep on my lap. What a cutie pie!'}

In [127]:
message = "Oh Jeez! This pizza is out of the world. Where did you get it? I wanna buy some more!"
response = llm_chain.invoke({"message":message})
json.loads(response)

{'sentiment': 'POSITIVE',
 'emotion': 'EXCITEMENT',
 'tone': 'ENTHUSIASTIC',
 'intent': 'INQUIRY',
 'action': 'PURCHASE',
 'summary': 'The speaker is excited about the pizza and wants to buy more.',
 'message': 'Oh Jeez! This pizza is out of the world. Where did you get it? I wanna buy some more!'}

In [128]:
prompt_template = """\
<|begin_of_text|><|start_header_id|>system<|end_header_id|>
You are an expert extraction algorithm who is keen of reading people through the message.
Only extract relavant information from the message.
If you do not know the value of an attribute asked to extract, return null for the attribute's value.

Here is a message: \n\n{message}<|eot_id|>
"""

prompt = PromptTemplate(
    template=prompt_template,
    input_variables=["message"],
    # partial_variables={"format_instructions": parser.get_format_instructions()},
)

llm_chain = prompt | llm

message = "I can't stand you any more. The way you act is extremely unacceptable!! I don't wanna talk to you any more"
response = llm_chain.invoke({"message":message})
# json.loads(response)
print(response)

After analyzing the message, I extracted the following relevant information:

* Sentiment: Negative
* Reason for negative sentiment: Unacceptable behavior
* Requested action: Don't want to talk to the person anymore


In [129]:
message = "This pug is crazy, haha. He always comes up to me and sleep on my lap. What a cutie pie!"
response = llm_chain.invoke({"message":message})
# json.loads(response)
print(response)

After analyzing the message, I extracted the following relevant information:

* Breed of dog: Pug
* Behavior: Comes up to the speaker and sleeps on their lap
* Adjective used to describe the dog: Crazy, Cutie Pie


# Play Ground